In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

C:\Users\z\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_hdf('../act_3.h5')
df.head().append(df.tail())

,item_id,course_id,video_id,watching_count,video_duration,local_watching_time,video_progress_time,video_start_time,video_end_time,local_start_time,local_end_time,drop
0,U_2461120,TsinghuaX+30240184+sp,V_102d2536030d47b4b343c3848f6f50b6,1,233.0,186,233.000000,0.00,233.000000,2019-04-04 21:13:20,2019-04-04 21:16:26,0
1,U_2461120,TsinghuaX+30240184+sp,V_4be5b36da31d4d15bc4afdd5a5cc1658,1,288.0,236,287.280000,0.72,288.000000,2019-04-04 21:20:35,2019-04-04 21:24:31,0
2,U_2461120,TsinghuaX+00310222X+sp,V_4c914a96924c4ba2b6ecaa4f58336892,1,767.0,774,767.000000,0.00,767.000000,2016-11-28 19:05:59,2016-11-28 19:18:53,0
3,U_2461120,TsinghuaX+30240184+sp,V_4f184a3de72d418caccbd3fa8624d5b6,3,209.0,613,209.000000,0.00,209.000000,2019-04-04 20:58:06,2019-04-04 21:08:35,0
4,U_2461120,TsinghuaX+00310222X+sp,V_75f9b0188a2648358a469aee137f07be,2,1132.0,105,104.230011,0.00,231.550003,2016-12-02 09:49:02,2016-12-02 09:50:47,1
35,U_8965811,TsinghuaX+40260092X+sp,V_d8b0299c16d24a23976f53602d91af1f,3,394.0,577,392.699982,0.00,393.149994,2018-10-17 20:37:36,2018-10-17 20:47:43,0
36,U_8965811,TsinghuaX+40260092X+sp,V_db213371577843f0a85b6fde8298beb1,3,102.0,72,101.720001,0.01,102.000000,2018-10-17 20:26:31,2018-10-17 20:27:52,0
37,U_8965811,TsinghuaX+00740043X_2015_T2+sp,V_eabc79addb914475ae1e4ad9c22f185b,2,143.0,102,143.000000,0.00,143.000000,2018-10-18 17:01:49,2018-10-18 17:05:31,0
38,U_8965811,TsinghuaX+40260092X+sp,V_f22a3ab6dcac48e685413d299ac119cb,2,76.0,114,73.849998,0.00,73.849998,2018-10-17 20:33:08,2018-10-17 20:35:57,1
39,U_8965811,TsinghuaX+00740043X_2015_T2+sp,V_f99fa9c9d7254b988cf82830cc72cb63,10,570.0,513,594.690018,0.00,570.000000,2018-10-18 17:06:00,2018-10-18 17:18:46,1


In [3]:
df.shape

(3400320, 12)

In [4]:
df.drop_duplicates(['item_id', 'course_id', 'video_id']).shape

(3385839, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3400320 entries, 0 to 39
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   item_id              object 
 1   course_id            object 
 2   video_id             object 
 3   watching_count       int64  
 4   video_duration       float64
 5   local_watching_time  int64  
 6   video_progress_time  float64
 7   video_start_time     float64
 8   video_end_time       float64
 9   local_start_time     object 
 10  local_end_time       object 
 11  drop                 int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 337.3+ MB


In [6]:
df['local_start_time'] = pd.to_datetime(df['local_start_time'])
df['local_end_time'] = pd.to_datetime(df['local_end_time'])
gc.collect()

18

In [7]:
uid1 = ['item_id', 'course_id', 'video_id']
uid2 = ['item_id', 'course_id']

In [8]:
data = df.groupby(uid1)['drop'].apply(list)
data = data.reset_index()
data['drop'] = data['drop'].map(lambda x: 1 if 1 in x else 0)
gc.collect()


# video_start_time, video_end_time, local_start_time, local_end_time
tmp = df.groupby(uid1, as_index=False).agg({
    'watching_count': np.sum,
    'video_duration': np.max,
    'local_watching_time': np.sum,
    'video_progress_time': np.sum,
    'video_start_time': np.min,
    'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max
})
data = data.merge(tmp, on=uid1, how='left')
del tmp
gc.collect()

0

In [9]:
data.head()

,item_id,course_id,video_id,drop,watching_count,video_duration,local_watching_time,video_progress_time,video_start_time,video_end_time,local_start_time,local_end_time
0,U_10000111,TsinghuaX+20350033X+sp,V_29ad964f79734a0e96a9e1661787b3f0,0,3,464.0,477,475.059967,0.0,464.0,2019-01-31 16:00:34,2019-01-31 16:09:09
1,U_10000111,TsinghuaX+20350033X+sp,V_37f7c9b491f142b3a1bb6b6c5ffd5e62,0,1,269.0,277,269.000000,0.0,269.0,2019-01-31 16:13:37,2019-01-31 16:18:14
2,U_10000111,TsinghuaX+20350033X+sp,V_51fb3df8572840b89d3b6496bbe3e68b,0,1,251.0,252,251.000000,0.0,251.0,2019-01-31 15:46:35,2019-01-31 15:50:47
3,U_10000111,TsinghuaX+20350033X+sp,V_bfb49368e2e24e15b342492fce69723b,0,4,284.0,394,386.320007,0.0,284.0,2019-01-31 15:35:43,2019-01-31 15:46:14
4,U_10000111,TsinghuaX+20350033X+sp,V_cee1aa84c8814b76936051a23ee9a074,0,1,296.0,297,296.000000,0.0,296.0,2019-01-31 16:19:04,2019-01-31 16:24:01


In [10]:
data.shape

(3385839, 12)

In [11]:
# 最晚的进度条时间点-最早的进度条时间点
data['video_watching_duration'] = data['video_end_time'] - data['video_start_time']

# 观看结束时间-观看开始时间
data['local_watching_duration'] = data.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

In [12]:
# 进度条是否读到最后，小于0
data['video_is_end'] = data['video_end_time'] - data['video_duration']

# 一开始是否拖到进度条
data['video_is_start'] = data['video_start_time'].map(lambda x: 1 if x > 0 else 0)

# 进度条是否从开始读到最后，小于0
data['video_is_start_end'] = data['video_watching_duration'] - data['video_duration']

# 进度条拖动的秒速 - 暂停时长
data['forward_seconds'] = data['video_watching_duration'] - data['video_progress_time']

# 暂停时长，
# df[''] = df[''] - df['']

# 实际观看时长是否有差误
data['local_watching_diff'] = data['local_watching_duration'] - data['local_watching_time']

# 倍速
data['speed'] = data['local_watching_time'] / (data['video_progress_time'] + 0.001)

In [13]:
data.columns

Index(['item_id', 'course_id', 'video_id', 'drop', 'watching_count',
       'video_duration', 'local_watching_time', 'video_progress_time',
       'video_start_time', 'video_end_time', 'local_start_time',
       'local_end_time', 'video_watching_duration', 'local_watching_duration',
       'video_is_end', 'video_is_start', 'video_is_start_end',
       'forward_seconds', 'local_watching_diff', 'speed'],
      dtype='object')

In [14]:
data.drop(['video_start_time', 'video_end_time', 'video_id'], axis=1, inplace=True)
gc.collect()

99

In [15]:
data['drop'].mean()

0.05680128322699337

In [16]:
data2 = data.groupby(uid2)['drop'].apply(list)
data2 = data2.reset_index()
data2['drop'] = data2['drop'].map(lambda x: 1 if 1 in x else 0)
gc.collect()
print(data2['drop'].mean())

0

In [ ]:
# video_start_time, video_end_time, local_start_time, local_end_time
tmp = data.groupby(uid2, as_index=False).agg({
    # 'watching_count': np.sum,
    # 'video_duration': np.sum,
    # 'local_watching_time': np.sum,
    # 'video_progress_time': np.sum,
    # 'video_start_time': np.min,
    # 'video_end_time': np.max,
    'local_start_time': np.min,
    'local_end_time': np.max,
    'video_is_end': np.sum,
    'video_is_start': np.mean,
    'video_is_start_end': np.sum,
    'forward_seconds': np.sum,
    'local_watching_diff': np.mean,
    'speed': np.mean
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

In [17]:
# watching_count
tmp = data.groupby(uid2, as_index=False)['watching_count'].agg({
    'watching_count_sum': 'sum',
    'watching_count_count': 'count',
    'watching_count_mean': 'mean',
    'watching_count_max': 'max',
    'watching_count_min': 'min',
    'watching_count_std': 'std',
    'watching_count_skew': 'skew',
    # 'watching_count_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [18]:
# video_duration
tmp = data.groupby(uid2, as_index=False)['video_duration'].agg({
    'video_duration_sum': 'sum',
    'video_duration_count': 'count',
    'video_duration_mean': 'mean',
    'video_duration_max': 'max',
    'video_duration_min': 'min',
    'video_duration_std': 'std',
    'video_duration_skew': 'skew',
    # 'video_duration_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [19]:
# local_watching_time
tmp = data.groupby(uid2, as_index=False)['local_watching_time'].agg({
    'local_watching_time_sum': 'sum',
    'local_watching_time_count': 'count',
    'local_watching_time_mean': 'mean',
    'local_watching_time_max': 'max',
    'local_watching_time_min': 'min',
    'local_watching_time_std': 'std',
    'local_watching_time_skew': 'skew',
    # 'local_watching_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [20]:
# video_progress_time
tmp = data.groupby(uid2, as_index=False)['video_progress_time'].agg({
    'video_progress_time_sum': 'sum',
    'video_progress_time_count': 'count',
    'video_progress_time_mean': 'mean',
    'video_progress_time_max': 'max',
    'video_progress_time_min': 'min',
    'video_progress_time_std': 'std',
    'video_progress_time_skew': 'skew',
    # 'video_progress_time_mad': 'mad'
})
data2 = data2.merge(tmp, on=uid2, how='left')
del tmp
gc.collect()

0

In [21]:
# 课程：观看结束时间-观看开始时间
data2['local_watching_duration'] = data2.apply(lambda x: (x['local_end_time'] - x['local_start_time']).seconds, axis=1)

data2['local_watching_diff'] = data2['local_watching_duration'] - data2['local_watching_time_sum']

In [22]:
data2.drop(['local_start_time', 'local_end_time'], axis=1, inplace=True)
del data
gc.collect()

20

In [23]:
data2['drop'].mean()

1.0

In [24]:
data2.columns

Index(['item_id', 'course_id', 'drop', 'video_is_end', 'video_is_start',
       'video_is_start_end', 'forward_seconds', 'local_watching_diff', 'speed',
       'watching_count_sum', 'watching_count_count', 'watching_count_mean',
       'watching_count_max', 'watching_count_min', 'watching_count_std',
       'watching_count_skew', 'video_duration_sum', 'video_duration_count',
       'video_duration_mean', 'video_duration_max', 'video_duration_min',
       'video_duration_std', 'video_duration_skew', 'local_watching_time_sum',
       'local_watching_time_count', 'local_watching_time_mean',
       'local_watching_time_max', 'local_watching_time_min',
       'local_watching_time_std', 'local_watching_time_skew',
       'video_progress_time_sum', 'video_progress_time_count',
       'video_progress_time_mean', 'video_progress_time_max',
       'video_progress_time_min', 'video_progress_time_std',
       'video_progress_time_skew', 'local_watching_duration'],
      dtype='object')

In [25]:
data2.to_csv('../../input/act_3_label.csv', index=False)
data2.to_hdf('../../input/act_3_label.h5', 'df', index=False)